# ATMS 523
## Module 4 Lecture 2

### Using a Web API to obtain data

This example shows you how to use a web API to obtain data from NCEI.  It is the same dataset we obtained within Module 3, but here we are making small calls to get only the data we want.

In [ ]:
#needed to make web requests
import requests

#store the data we get as a dataframe
import pandas as pd

#convert the response as a strcuctured json
import json

#mathematical operations on lists
import numpy as np

#parse the datetimes we get from NOAA
from datetime import datetime

#import time for sleeping
import time

#add the access token you got from NOAA # this is snesbitt's so don't mess with it!
Token = 'qGhKPoTezWscLZFQAzmnmBzBkrsiwroe'

#Enter data type and station ID from inventory - here is GHCND https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt
station_id = 'GHCND:USC00118740'

In [ ]:
#initialize lists to store data
dates_mintemp = []
dates_maxtemp = []
dates_precip = []
min_temps = []
max_temps = []
precip = []

#for each year from 1905-2022 where we know we have data inventory ...
for year in range(1905, 2023):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMIN&datatypeid=TMAX&datatypeid=PRCP&limit=1000&stationid='+station_id+'&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #TODO check for empty request

    #get all items in the response which are max&min temperature readings
    maxtemps = [item for item in d['results'] if item['datatype']=='TMAX']
    mintemps = [item for item in d['results'] if item['datatype']=='TMIN']
    precips = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all average temperature readings
    dates_maxtemp += [item['date'] for item in maxtemps]
    dates_mintemp += [item['date'] for item in mintemps]
    dates_precip += [item['date'] for item in precips]
    #get the actual temperature from the returned data
    max_temps += [item['value'] for item in maxtemps]
    min_temps += [item['value'] for item in mintemps]
    precip += [item['value'] for item in precips]
    time.sleep(0.2) # API max 5 requests per second


In [ ]:
#initialize dataframe
df_temp_min = pd.DataFrame()
df_temp_max = pd.DataFrame()
df_precip = pd.DataFrame()

#populate date and min and max temperature & precip fields (convert string date to datetime)
df_temp_min['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_mintemp]
df_temp_min['minTemp'] = [float(v)/10.0 for v in min_temps]

df_temp_max['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_maxtemp]
df_temp_max['maxTemp'] = [float(v)/10.0 for v in max_temps]

df_precip['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_precip]
df_precip['precip'] = [float(v)/10.0 for v in precip]


In [ ]:
#merge the dataframes
newdf_all = pd.merge(df_temp_max,df_temp_min, left_index=True, right_index=True)
newdf_all = pd.merge(newdf_all, df_precip, left_index=True, right_index=True)
newdf_all.drop(columns = ['date_x', 'date_y'], inplace=True)
newdf_all = newdf_all.set_index('date')

In [ ]:
newdf_all